In [2]:
#import dependencies
import pandas as pd
import spotipy
from config import CLIENT_ID, CLIENT_SECRET
import requests
import json

In [26]:
# read in chart csv
data = pd.read_csv("Data/billboard_hot100_2016.csv")
chart_df = pd.DataFrame(data)
chart_df.head()

#add new row in chart df 
chart_df["Spotify_Artist"] = ""
chart_df.head()

# trim down dataframe by dropping duplicate songs and reset index
unique_songs_df = chart_df.drop_duplicates(subset=["Title", "Artist"])
unique_songs_df.reset_index(inplace=True, drop=True)
unique_songs_df.head(200)

,Date,Title,Artist,Rank,IsNew,Spotify_Artist
0,2016-01-09,Hello,Adele,1,False,
1,2016-01-09,Sorry,Justin Bieber,2,False,
2,2016-01-09,Love Yourself,Justin Bieber,3,False,
3,2016-01-09,Hotline Bling,Drake,4,False,
4,2016-01-09,What Do You Mean?,Justin Bieber,5,False,
...,...,...,...,...,...,...
195,2016-04-02,Fast Car,Jonas Blue Featuring Dakota,100,True,
196,2016-04-09,Team,Iggy Azalea,42,True,
197,2016-04-09,Be Alright,Ariana Grande,43,True,
198,2016-04-09,Body,Dreezy Featuring Jeremih,98,True,


In [18]:
AUTH_URL = 'https://accounts.spotify.com/api/token'
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
token = auth_response_data['access_token']

sp = spotipy.Spotify(token)

In [27]:
#testing
#5s91S8hSrJXqYdhwHOEioV "You Can Have It All" - "Yo La Tengo"
artist= """Yo La Tengo"""
track= """You Can Have It All"""
track_info = sp.search(q='artist:' + artist + ' track:' + track, type='track')
#print(json.dumps(track_id, indent=4, sort_keys=True))
track_id = track_info['tracks']['items'][0]['id']
track_id

'5s91S8hSrJXqYdhwHOEioV'

In [29]:
#Iterates through chart and formats artist names for spotify api
for index, row in unique_songs_df.iterrows():
    no_feat = row["Artist"].split(" Featuring")[0]
    first_artist = no_feat.split(" &")[0]
    unique_songs_df.iat[index, 5] = first_artist

unique_songs_df.head()

,Date,Title,Artist,Rank,IsNew,Spotify_Artist
0,2016-01-09,Hello,Adele,1,False,Adele
1,2016-01-09,Sorry,Justin Bieber,2,False,Justin Bieber
2,2016-01-09,Love Yourself,Justin Bieber,3,False,Justin Bieber
3,2016-01-09,Hotline Bling,Drake,4,False,Drake
4,2016-01-09,What Do You Mean?,Justin Bieber,5,False,Justin Bieber


In [34]:
# Creates songs data dict, iterates through chart and searches spotify api for spotify ID
songs_data = {"Artist": [], "Track": [], "Track_ID": []}
data_not_found = {"Artist": [], "Track": [], "Track_ID": []}
for index, row in unique_songs_df.iterrows():
    artist= row["Spotify_Artist"]
    track= row["Title"]
    try:
        track_info = sp.search(q='artist:' + artist + ' track:' + track, type='track')
        track_id = track_info['tracks']['items'][0]['id']
    except:
        track_id = "not found"
        data_not_found["Artist"].append(artist)
        data_not_found["Track"].append(track)
        data_not_found["Track_ID"].append(track_id)
        print(track + " by " + artist + " not found")

    songs_data["Artist"].append(artist)
    songs_data["Track"].append(track)
    songs_data["Track_ID"].append(track_id)


Don't by Bryson Tiller not found
Can't Feel My Face by The Weeknd not found
Me, Myself & I by G-Eazy x Bebe Rexha not found
I'll Show You by Justin Bieber not found
Don't Let Me Down by The Chainsmokers not found
Ain't Your Mama by Jennifer Lopez not found
You Don't Own Me by Grace not found
Don't Hurt Yourself by Beyonce not found
Don't Mind by Kent Jones not found
Can't Stop The Feeling! by Justin Timberlake not found
With You Tonight / Hasta El Amanecer by Nicky Jam not found
Nothing Is Promised by Mike WiLL Made-It x Rihanna not found
Broccoli by D.R.A.M. not found
Ain't No Stopping Us Now by Kane Brown not found
Love Make The World Go Round by Jennifer Lopez not found
Duele El Corazon by Enrique Iglesias Feat. Wisin or Tinashe not found
Pick Up The Phone by Young Thug And Travis Scott not found
Rock On by Tucker Beathard not found
Don't Touch My Hair by Solange not found
Don't Wanna Know by Maroon 5 not found
Ain't My Fault by Zara Larsson not found
Bad Things by Machine Gun Kelly

In [39]:
#creates dataframe for songs WITH AND WITHOUT track IDs
songs_df = pd.DataFrame(songs_data)
songs_df.head()

,Artist,Track,Track_ID
0,Adele,Hello,62PaSfnXSMyLshYJrlTuL3
1,Justin Bieber,Sorry,09CtPGIpYB4BrO8qb1RGsF
2,Justin Bieber,Love Yourself,50kpGaPAhYJ3sGmk6vplg0
3,Drake,Hotline Bling,0wwPcA6wtMf6HUMpIRdeP7
4,Justin Bieber,What Do You Mean?,4B0JvthVoAAuygILe3n4Bs


In [40]:
#creates dataframe for songs WITHOUT track IDs
no_data_df = pd.DataFrame(data_not_found)
no_data_df.head()

,Artist,Track,Track_ID
0,Bryson Tiller,Don't,not found
1,The Weeknd,Can't Feel My Face,not found
2,G-Eazy x Bebe Rexha,"Me, Myself & I",not found
3,Justin Bieber,I'll Show You,not found
4,The Chainsmokers,Don't Let Me Down,not found


In [41]:
# Turn songs dataframe into csv
songs_df.to_csv("Data/spotify_ids_2016.csv", index=False)